In [ ]:
from sklearn.preprocessing import LabelBinarizer as LB
from sklearn.preprocessing import normalize 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.layers.core import Reshape, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense , BatchNormalization, Activation
from keras import metrics

from pandas import DataFrame as df

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import numpy as np

import tarfile
import pickle
import random
import keras
import sys
import gc

Get data

In [ ]:
!wget http://opendata.deepsig.io/datasets/2016.10/RML2016.10b.tar.bz2
!tar jxf RML2016.10b.tar.bz2

--2021-01-14 13:38:54--  http://opendata.deepsig.io/datasets/2016.10/RML2016.10b.tar.bz2
Resolving opendata.deepsig.io (opendata.deepsig.io)... 52.14.91.165
Connecting to opendata.deepsig.io (opendata.deepsig.io)|52.14.91.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1245608913 (1.2G) [application/x-bzip2]
Saving to: ‘RML2016.10b.tar.bz2’

RML2016.10b.tar.bz2 100%[===================>]   1.16G  15.2MB/s    in 1m 49s  

2021-01-14 13:40:43 (10.9 MB/s) - ‘RML2016.10b.tar.bz2’ saved [1245608913/1245608913]



In [ ]:
file = open("RML2016.10b.dat",'rb')
Xd = pickle.load(file, encoding = 'bytes')
snrs, mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = [] 
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)
file.close()

## Create Features Space

In [ ]:
features = {}

In [ ]:
features['raw'] = X[:,0], X[:,1]

In [ ]:
features['derivative'] = normalize(np.gradient(X[:,0], axis = 1)), normalize(np.gradient(X[:,1], axis = 1))

In [ ]:
features['integral'] = normalize(np.cumsum(X[:,0], axis = 1)), normalize(np.cumsum(X[:,1], axis = 1))

**All Togetheer Feature Space**

In [ ]:
def extract_features(*arguments):
    
    desired = ()            
    for arg in arguments:
        desired += features[arg]
    
    return np.stack(desired, axis = 1)

## Train and Test Data Split

In [ ]:
data = extract_features('raw')
labels = np.array(lbl)

In [ ]:
in_shape = data[0].shape
out_shape = tuple([1]) + in_shape

In [ ]:
in_shape

(2, 128)

In [ ]:
np.random.seed(10)

n_examples = labels.shape[0]

r = np.random.choice(range(n_examples), n_examples, replace = False)

train_examples = r[:n_examples//2]
test_examples =  r[n_examples//2:]

In [ ]:
X_train = data[train_examples]
X_test = data[test_examples]

y_train = LB().fit_transform(labels[train_examples][:,0])
y_test = LB().fit_transform(labels[test_examples][:,0])

snr_train = labels[train_examples][:,1].astype(int)
snr_test = labels[test_examples][:,1].astype(int)

## ABRO Model for Convolution Connected Neural Networks

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def modelCNN(btch_norm = False):
  dr = 0.5
  model = Sequential()
  model.add(Reshape(out_shape, input_shape = in_shape))
  model.add(ZeroPadding2D((0, 2), data_format = 'channels_first'))
  model.add(Conv2D(256, (1, 3), padding = 'valid', name="conv1", kernel_initializer='glorot_uniform', data_format="channels_first"))

  if (btch_norm):
    model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(Dropout(dr))
  model.add(ZeroPadding2D((0,2), data_format = 'channels_first'))
  model.add(Conv2D(80, (2, 3), activation="relu", name="conv3", padding="valid", kernel_initializer="glorot_uniform", data_format="channels_first"))

  if (btch_norm):
    model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(Dropout(dr))
  model.add(Flatten())   
  model.add(Dense(256, activation="relu", name="dense1", kernel_initializer="he_normal"))

  if (btch_norm):
    model.add(BatchNormalization())
  model.add(Activation("relu"))
  model.add(Dropout(dr))
  model.add(Dense(10, name="dense3", kernel_initializer="he_normal", activation = 'softmax'))
  model.add(Reshape([len(mods)]))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
  model.summary()
  return model

In [ ]:
model = modelCNN()
history = model.fit(X_train, y_train, epochs = 80, batch_size = 1024, validation_split = 0.05, callbacks=[EarlyStopping(patience = 15, restore_best_weights = True)])

In [ ]:
model.save("cnn.h5")

In [ ]:
model_batch = modelCNN(True)
history_batch = model_batch.fit(X_train, y_train, epochs = 80, batch_size = 1024, validation_split = 0.05, callbacks=[EarlyStopping(patience = 15, restore_best_weights = True)])

In [ ]:
model_batch.save("cnn_batch.h5")

## Evaluation

In [ ]:
y_pred_cnn = model.predict(X_test)

In [ ]:
y_pred_cnn_batch = model_batch.predict(X_test)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm, annot=True, )

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm_batch, annot=True)